# 拓扑排序简介

> Topological Sorting: 一种对有向无环图（DAG）的所有顶点进行线性排序的方法，使得图中任意一点u和v，如果存在有向边$<u,v>$，则u必须在v之前出现。对有向图进行拓扑排序产生的线性序列称为满足拓扑次序的序列，简称拓扑排序。

图的拓扑排序是针对有向无环图（DAG）来说的，无向图和有向有环图没有拓扑排序，或者说不存在拓扑排序。

![有向无环图](../../../image/有向无环图.png)

如上图中的有向无环图（DAG）所示，$v_1 \rightarrow v_2 \rightarrow v_3 \rightarrow v_4 \rightarrow v_5 \rightarrow v_6$ 是该图的一个拓扑序列。与此同时，$v_1 \rightarrow v_2 \rightarrow v_3 \rightarrow v_4 \rightarrow v_6 \rightarrow v_5$ 也是该图的一个拓扑序列。也就是说，对于一个有向无环图来说，拓扑序列可能不止一个。

# 拓扑排序的实现方法

拓扑排序有两种实现方法，分别是「Kahn 算法」和「DFS 深度优先搜索算法」。

## Kahn算法

### Kahn算法基础思想

> 1. 不断找寻有向图中入度为0的顶点，将其输出。
> 2. 然后删除入度为0的顶点和从该顶点出发的有向边。
> 3. 重复上述操作直到图为空，或者找不到入度为0的节点为止。

### 实现步骤

1. 使用数组$indegrees$记录图中各个顶点的入度。
2. 维护一个入度为0的顶点集合$S$（可使用栈、队列、优先队列）。
3. 每次从集合中选择任何一个没有前驱（即入度为0）的顶点u，将其输出到拓扑序列$order$中。
4. 从图中删除该顶点u，并且删除从该顶点出发的有向边$<u,v>$（也就是把该顶点可达的顶点入度都减1）。如果删除该边后顶点v的入度变为0，则将顶点v放入集合$S$中。
5. 重复上述过程，直到集合$S$为空，或者图中还有顶点未被访问（说明一定存在环路，无法形成拓扑序列）。
6. 如果不存在环路，则$order$中顶点的顺序为拓扑排序结果。

### 实现代码

In [1]:
import collections

class Solution:
    # 拓扑排序，graph 中包含所有顶点的有向边关系（包括无边顶点）
    def topologicalSortingKahn(self, graph: dict):
        indegrees = {u: 0 for u in graph}   # indegrees 用于记录所有顶点入度
        for u in graph:
            for v in graph[u]:
                indegrees[v] += 1           # 统计所有顶点入度
        
        # 将入度为 0 的顶点存入集合 S 中
        S = collections.deque([u for u in indegrees if indegrees[u] == 0])
        order = []                          # order 用于存储拓扑序列
        
        while S:
            u = S.pop()                     # 从集合中选择一个没有前驱的顶点 0
            order.append(u)                 # 将其输出到拓扑序列 order 中
            for v in graph[u]:              # 遍历顶点 u 的邻接顶点 v
                indegrees[v] -= 1           # 删除从顶点 u 出发的有向边
                if indegrees[v] == 0:       # 如果删除该边后顶点 v 的入度变为 0
                    S.append(v)             # 将其放入集合 S 中
        
        if len(indegrees) != len(order):    # 还有顶点未遍历（存在环），无法构成拓扑序列
            return []
        return order                        # 返回拓扑序列
    
    
    def findOrder(self, n: int, edges):
        # 构建图
        graph = dict()
        for i in range(n):
            graph[i] = []
            
        for u, v in edges:
            graph[u].append(v)
            
        return self.topologicalSortingKahn(graph)


## 基于DFS实现算法

### 基于 DFS 实现拓扑排序算法的基本思想

> 1. 对于一个顶点u，深度优先遍历从该顶点出发的有向边$<u,v>$。如果从该顶点u出发的所有相邻顶点v都已经搜索完毕，则回溯到顶点u时，该顶点u应该位于其所有相邻顶点v的前面（拓扑序列中）。
> 2. 这样一来，当我们对每个顶点进行深度优先搜索，在回溯到该顶点时将其放入栈中，则最终从栈顶到栈底的序列就是一种拓扑排序。

### 实现步骤

1. 使用集合$visited$用于记录当前顶点是否被访问过，避免重复访问。
2. 使用集合$onStack$用于记录同一次深度优先搜索时，当前顶点是否被访问过。如果当前顶点被访问过，则说明图中存在环路，无法构成拓扑序列。
3. 使用布尔变量$hasCycle$用于判断图中是否存在环。
4. 从任意一个未被访问的顶点u出发。
    1. 如果顶点u在同一次深度优先搜索时被访问过，则说明存在环。
    2. 如果当前顶点被访问或者有环时，则无需再继续遍历，直接返回。
5. 将顶点u标记为被访问过，并在本次深度优先搜索中标记为访问过。然后深度优先遍历从顶点u出发的有向边$<u,v>$。
6. 当顶点u的所有相邻顶点v都被访问后，回溯前记录当前节点u（将当前节点u输出到拓扑序列$order$中）。
7. 取消本次深度优先搜索时，顶点u的访问标记。
8. 对其他未被访问的顶点重复4~7步过程，直到所有节点都遍历完，或者出现环。
9. 如果不存在环路，则将$order$逆序排序后，顶点的顺序就是拓扑排序的结果。

### 实现代码

In [2]:
import collections

class Solution:
    # 拓扑排序，graph 中包含所有顶点的有向边关系（包括无边顶点）
    def topologicalSortingDFS(self, graph: dict):
        visited = set()                     # 记录当前顶点是否被访问过
        onStack = set()                     # 记录同一次深搜时，当前顶点是否被访问过
        order = []                          # 用于存储拓扑序列
        hasCycle = False                    # 用于判断是否存在环
        
        def dfs(u):
            nonlocal hasCycle
            if u in onStack:                # 同一次深度优先搜索时，当前顶点被访问过，说明存在环
                hasCycle = True
            if u in visited or hasCycle:    # 当前节点被访问或者有环时直接返回
                return
            
            visited.add(u)                  # 标记节点被访问
            onStack.add(u)                  # 标记本次深搜时，当前顶点被访问
    
            for v in graph[u]:              # 遍历顶点 u 的邻接顶点 v
                dfs(v)                      # 递归访问节点 v
                    
            order.append(u)                 # 后序遍历顺序访问节点 u
            onStack.remove(u)               # 取消本次深搜时的 顶点访问标记
        
        for u in graph:
            if u not in visited:
                dfs(u)                      # 递归遍历未访问节点 u
        
        if hasCycle:                        # 判断是否存在环
            return []                       # 存在环，无法构成拓扑序列
        order.reverse()                     # 将后序遍历转为拓扑排序顺序
        return order                        # 返回拓扑序列
    
    def findOrder(self, n: int, edges):
        # 构建图
        graph = dict()
        for i in range(n):
            graph[i] = []
        for v, u in edges:
            graph[u].append(v)
        
        return self.topologicalSortingDFS(graph)
